In [19]:
import os
import numpy as np
import itertools
import time
from scipy.spatial import distance
from scipy.optimize import fsolve
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [20]:
class PolynomialRegression(object):
    def __init__(self, degree=3, coeffs=None):
        self.degree = degree
        self.coeffs = coeffs

    def fit(self, X, y):
        self.coeffs = np.polyfit(X.ravel(), y, self.degree)

    def get_params(self, deep=False):
        return {'coeffs': self.coeffs}

    def set_params(self, coeffs=None, random_state=None):
        self.coeffs = coeffs

    def predict(self, X):
        poly_eqn = np.poly1d(self.coeffs)
        y_hat = poly_eqn(X.ravel())
        return y_hat

    def score(self, X, y):
        return mean_squared_error(y, self.predict(X))

In [21]:
data_folder = '/Users/jeffeehsiung/Desktop/master thesis/Seoul_Robotics_lane_detection/processed_lidar'
lidar_files = sorted(os.listdir(data_folder))
lidar_fpath = [os.path.join(data_folder, f) for f in lidar_files]
bin = 0
lidar_bin = lidar_fpath[bin]
lidar_data = np.fromfile(lidar_bin, dtype=np.float32).reshape(-1, 3) # 3 columns for x, y, z
# Find the min and max x-coordinates from the lidar_data
min_x = np.min(lidar_data[:, 0])
max_x = np.max(lidar_data[:, 0])

# Adjust the x_list to cover the full range of x-coordinates in the point cloud
# Depending on your requirements, you might want to round min_x and max_x to the nearest integer
# or adjust them slightly to ensure they encompass the entire range of interest
min_x = np.floor(min_x).astype(int)  # Optional: rounding down to the nearest integer
max_x = np.ceil(max_x).astype(int)   # Optional: rounding up to the nearest integer

In [22]:
grid_dict = {}
for y in (-1,1):
    grid_list = [] 
    max_lane_width = 3.9
    line_of_sight = 10
    for x in range(min_x, max_x + 1):
        if y == -1:
            grid_x_up_bound = x * line_of_sight
            grid_x_low_bound = (x-1) * line_of_sight
            grid_y_up_bound = (y+1) * max_lane_width
            grid_y_low_bound = y * max_lane_width
            grid = [grid_x_up_bound, grid_x_low_bound, grid_y_up_bound, grid_y_low_bound]
            coord = (y, x)
            grid_dict[coord] = grid
        else:
            grid_x_up_bound = x * line_of_sight
            grid_x_low_bound = (x-1) * line_of_sight
            grid_y_up_bound = y * max_lane_width
            grid_y_low_bound = (y-1) * max_lane_width
            grid = [grid_x_up_bound, grid_x_low_bound, grid_y_up_bound, grid_y_low_bound]
            coord = (y, x)
            grid_dict[coord] = grid

lidar_data[:, 2] = 0

data_in_grid = {}
for grid_cell_index, grid_cell_coord in enumerate(grid_dict):
    x_upper_bound, x_lower_bound, y_upper_bound, y_lower_bound = grid_dict[grid_cell_coord]
    inGrid_lidar_data = np.delete(lidar_data, np.where(
    (lidar_data[:, 0] > x_upper_bound) |
    (lidar_data[:, 0] <= x_lower_bound) |
    (lidar_data[:, 1] > y_upper_bound) |
    (lidar_data[:, 1] <= y_lower_bound))[0], axis = 0)
    
    data_in_grid[grid_cell_coord] = inGrid_lidar_data

In [23]:
import numpy as np
from scipy.optimize import fsolve
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error

def cost(up_lane_coeffs, down_lane_coeffs, x_range):
    """
    Calculate the cost as the mean squared error between the true lane width (assumed constant)
    and the measured lane width across a range of x-values.

    Args:
    - up_lane_coeffs (array): Coefficients for the polynomial representing the upper lane boundary.
    - down_lane_coeffs (array): Coefficients for the polynomial representing the lower lane boundary.
    - x_range (array): Range of x-values to evaluate the lane width.

    Returns:
    - float: The cost calculated as mean squared error.
    """
    # Define polynomial functions for the upper and lower lanes
    poly_up = np.poly1d(up_lane_coeffs)
    poly_down = np.poly1d(down_lane_coeffs)

    # Calculate derivatives for the lower lane polynomial
    poly_down_deriv = np.polyder(poly_down)

    # Vectorized calculation of y-values for lower and upper lanes
    y_down = poly_down(x_range)
    y_deriv_down = poly_down_deriv(x_range)

    # Initialize measured intervals
    interval_measured = []

    # Calculate perpendicular distances for each x in x_range
    for x, y, dy in zip(x_range, y_down, y_deriv_down):
        # Define the perpendicular line at (x, y) of the lower lane
        perp_slope = -1 / dy if dy != 0 else np.inf
        y_intercept = y - perp_slope * x

        # Define a function for the intersection with the upper lane
        def intersection_fun(x_intersect):
            return perp_slope * x_intersect + y_intercept - poly_up(x_intersect)

        # Use fsolve to find the intersection point
        x_intersect = fsolve(intersection_fun, x)[0]

        # Calculate the perpendicular distance
        y_intersect = perp_slope * x_intersect + y_intercept
        dist = distance.euclidean([x_intersect, y_intersect], [x, y])
        interval_measured.append(dist)

    # True interval is assumed to be constant (e.g., 3 meters)
    interval_truth = np.full_like(x_range, 3)

    # Calculate MSE as the cost
    cost = mean_squared_error(interval_truth, interval_measured)
    return cost

# Example usage:
# Assuming up_lane_coeffs and down_lane_coeffs are defined, and x_range is set
# cost_value = cost(up_lane_coeffs, down_lane_coeffs, np.linspace(min_x, max_x, num=100))


In [24]:
min_sample_threshold = 5
poly_degree = 3
# shape of lidar_data
n = lidar_data.shape[1]
data_repres_left = np.empty((0, n))
data_repres_right = np.empty((0, n)) 

iteration = 0
max_iter = 1000
prev_error = 1000

start = time.time()
while iteration <= max_iter:
    for y in (-1, 1):
        for x in range(min_x, max_x + 1):
            if y == -1:
                if len(data_in_grid[y, x]) >= min_sample_threshold:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = poly_degree)
                    data_repres_left = np.append(data_repres_left, data_in_grid[y, x][idx, :], axis = 0)
                elif len(data_in_grid[y, x]) == 0:
                    pass
                else:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = len(data_in_grid[y,x]))
                    data_repres_left = np.append(data_repres_left, data_in_grid[y, x][idx, :], axis = 0)

            elif y == 1:
                if len(data_in_grid[y, x]) >= min_sample_threshold:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = poly_degree)
                    data_repres_right = np.append(data_repres_right, data_in_grid[y, x][idx, :], axis = 0)
                elif len(data_in_grid[y, x]) == 0:
                    pass
                else:
                    idx = np.random.randint(len(data_in_grid[y, x]), size = len(data_in_grid[y,x]))
                    data_repres_right = np.append(data_repres_right, data_in_grid[y, x][idx, :], axis = 0)
                    
    left_grids_x = np.sort(data_repres_left, axis = 0)[:, 0] 
    left_grids_y = np.sort(data_repres_left, axis = 0)[:, 1]
    right_grids_x = np.sort(data_repres_right, axis = 0)[:, 0] 
    right_grids_y = np.sort(data_repres_right, axis = 0)[:, 1]

    ransac_up = RANSACRegressor(PolynomialRegression(degree = poly_degree),
                             min_samples = 7,    
                             max_trials = 10000,
                             random_state=0)
    ransac_up.fit(np.expand_dims(left_grids_x, axis=1), left_grids_y)
    left_grids_y_pred = ransac_up.predict(np.expand_dims(left_grids_x, axis=1))
    left_lane_coeffs= ransac_up.estimator_.get_params(deep = True)["coeffs"]

    ransac_down = RANSACRegressor(PolynomialRegression(degree = poly_degree), 
                             min_samples = 7,    
                             max_trials = 10000,
                             random_state=0)
    ransac_down.fit(np.expand_dims(right_grids_x, axis=1), right_grids_y)
    right_grids_y_pred = ransac_down.predict(np.expand_dims(right_grids_x, axis=1))
    right_lane_coeffs = ransac_down.estimator_.get_params(deep = True)["coeffs"]
    
    ego_lane_coeffs_pair = np.append(right_lane_coeffs, left_lane_coeffs, axis = 0) 
    curr_error = cost(left_lane_coeffs, right_lane_coeffs, np.linspace(min_x, max_x, num=100))      
    
    if curr_error < prev_error:
        prev_error = curr_error 
        best_coeffs_pair = ego_lane_coeffs_pair

    iteration += 1

print("time spent: {:.2f} sec".format(time.time() - start)) 
print(best_coeffs_pair)

# saved as txt file in the sample_output folder in the format of each line of four coefficients per time 
# ... rest of your code ...

# Convert the best_coeffs_pair to a 2D array with 4 columns
best_coeffs_pair = best_coeffs_pair.reshape(-1, 4)

# Save the coefficients to a text file, with each number formatted as a floating-point number
# get the name of file in pointclouds folder with extension .bin
# and replace the extension with .txt
data_folder = '/Users/jeffeehsiung/Desktop/master thesis/Seoul_Robotics_lane_detection/pointclouds'
lidar_files = sorted(os.listdir(data_folder))
lidar_fpath = [os.path.join(data_folder, f) for f in lidar_files]
lidar_bin = lidar_fpath[bin]
lidar_bin_name = os.path.basename(lidar_bin)
lidar_txt_name = lidar_bin_name.replace('.bin', '.txt')
# save the file in the sample_output folder
# and numbers separated by semicolons
np.savetxt(os.path.join('sample_output', lidar_txt_name), best_coeffs_pair, fmt='%.15e', delimiter=';')

# ... rest of your code ...


0.030358076095581055
1
0.12471413612365723
2
0.15663504600524902
3
0.18505096435546875
4
0.21193194389343262
5
0.24763894081115723
6
0.27587103843688965
7
0.30373597145080566
8
0.3275730609893799
9
0.353787899017334
10
0.38602709770202637
11
0.4192540645599365
12
0.44687795639038086
13
0.47540903091430664
14
0.5039398670196533
15
0.5274851322174072
16
0.5531280040740967
17
0.5747120380401611
18
0.5954849720001221
19
0.6176149845123291
20
0.6365621089935303
21
0.6577041149139404
22
0.6797850131988525
23
0.702415943145752
24
0.7238080501556396
25
0.7448759078979492
26
0.7657780647277832
27
0.7882940769195557
28
0.8086569309234619
29
0.8296270370483398
30
0.8525509834289551
31
0.8723940849304199
32
0.8916099071502686
33
0.9150309562683105
34
0.9361810684204102
35
0.9562408924102783
36
0.9771559238433838
37
0.9994590282440186
38
1.0202410221099854
39
1.0397489070892334
40
1.05948805809021
41
1.0809180736541748
42
1.1022028923034668
43
1.123520851135254
44
1.1460838317871094
45
1.1677930355

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ